In [ ]:
import pandas as pd
import shutil
import os
import numpy as np
import matplotlib.pyplot as plt

plt.rcParams['figure.dpi'] = 300
os.makedirs('img', exist_ok=True)
task = get_param_in_cwd('task_column') or 'label'
label_file = r'E:\Project_Result\diagnosis_UC_CD\oneAI\test_cohort\external-2-class\label.csv'
AI_file = r'E:\Project_Result\diagnosis_UC_CD\oneAI\test_cohort\external-2-class\AI.csv'
human_only_file = r'E:\Project_Result\diagnosis_UC_CD\oneAI\test_cohort\external-2-class\human_only.csv'
AI_assistant_file = r'E:\Project_Result\diagnosis_UC_CD\oneAI\test_cohort\external-2-class\ai_assist.csv'


labels = [task]
label_data_ = pd.read_csv(label_file)
label_data_ = label_data_.dropna(axis=1)

ids = label_data_['ID']
print(label_data_.columns)
label_data = label_data_[['ID'] + labels]
label_data

In [ ]:
import pandas as pd

subset = 'train'
ALL_results = pd.read_csv(AI_file)
ALL_results.columns = ['ID', '-0','AI']
human_results = pd.read_csv(human_only_file)
model_names = ['AI'] + list(human_results.columns[1:])
ALL_results = pd.merge(pd.merge(ALL_results, human_results, on='ID', how='inner'), label_data_)

ALL_results = ALL_results.dropna(axis=1)
ALL_results

In [ ]:
from matplotlib.markers import MarkerStyle

gt = [np.array(ALL_results[task]) for d in model_names]
pred_train = [np.array(ALL_results[d]) for d in model_names]
okcomp.comp1.draw_roc(gt, pred_train, labels=model_names, title=f"Model AUC", auto_point=True)
plt.savefig(f'human_AI/external_ibd_normal/Human_auc.svg')

In [ ]:
metric = []
for mname, y, score in zip(model_names, gt, pred_train):

    acc, auc, ci, tpr, tnr, ppv, npv, precision, recall, f1, thres = analysis_pred_binary(y, score, use_youden=False)
    ci = f"{ci[0]:.4f} - {ci[1]:.4f}"
    metric.append((mname, acc, auc, ci, tpr, tnr, ppv, npv, precision, recall, f1, thres, f"Without AI"))
a = pd.DataFrame(metric, index=None, columns=['Signature', 'Accuracy', 'AUC', '95% CI', 'Sensitivity', 'Specificity', 
                                          'PPV', 'NPV', 'Precision', 'Recall', 'F1','Threshold', 'Cohort'])
a

In [ ]:
a['group'] = a['Signature'].map(lambda x: x if x == 'AI' else x[:-1])
aa = a.groupby('group').mean().reset_index()
aa['Cohort'] = 'Without AI'
aa

In [ ]:

delong = []
delong_columns = []
this_delong = []
plt.figure(figsize=(15, 12))
cm = np.zeros((len(model_names), len(model_names)))
for i, mni in enumerate(model_names):
    for j, mnj in enumerate(model_names):
        if i <= j:
            cm[i][j] = np.nan
        else:
            cm[i][j] = delong_roc_test(ALL_results[task], ALL_results[mni], ALL_results[mnj])[0][0]
cm = pd.DataFrame(cm[1:, :-1], index=model_names[1:], columns=model_names[:-1])
draw_matrix(cm, annot=True, cmap='jet_r', cbar=True)
plt.title(f'Cohort {subset} Delong')
plt.savefig(f'human_AI/external_ibd_normal/delong_each_cohort_{subset}.svg', bbox_inches = 'tight')
plt.show()

In [ ]:
import pandas as pd

subset = 'test'
ALL_results = pd.read_csv(AI_file)
ALL_results.columns = ['ID', '-0', 'AI']
human_results = pd.read_csv(AI_assistant_file)
model_names = ['AI'] + list(human_results.columns[1:])
ALL_results = pd.merge(pd.merge(ALL_results, human_results, on='ID', how='inner'), label_data_)

ALL_results = ALL_results.dropna(axis=1)
ALL_results

In [ ]:
from matplotlib.markers import MarkerStyle

pred_column = [f'{task}-0', f'{task}-1']
gt = [np.array(ALL_results[task]) for d in model_names]
pred_train = [np.array(ALL_results[d]) for d in model_names]
okcomp.comp1.draw_roc(gt, pred_train, labels=model_names, title=f"Model AUC", auto_point=True)
plt.savefig(f'human_AI/external_ibd_normal/Human_auc_AI.svg')

In [ ]:
import analysis_pred_binary
metric = []
for mname, y, score in zip(model_names, gt, pred_train):
    # 计算验证集指标
    acc, auc, ci, tpr, tnr, ppv, npv, precision, recall, f1, thres = analysis_pred_binary(y, score, use_youden=False)
    ci = f"{ci[0]:.4f} - {ci[1]:.4f}"
    metric.append((mname, acc, auc, ci, tpr, tnr, ppv, npv, precision, recall, f1, thres, f"With AI"))
b = pd.DataFrame(metric, index=None, columns=['Signature', 'Accuracy', 'AUC', '95% CI', 'Sensitivity', 'Specificity', 
                                          'PPV', 'NPV', 'Precision', 'Recall', 'F1','Threshold', 'Cohort'])
b

In [ ]:
b['group'] = b['Signature'].map(lambda x: x if x == 'AI' else x[:-1])
ba = b.groupby('group').mean().reset_index()
ba['Cohort'] = 'With AI'
ba

In [ ]:

delong = []
delong_columns = []
this_delong = []
plt.figure(figsize=(15, 12))
cm = np.zeros((len(model_names), len(model_names)))
for i, mni in enumerate(model_names):
    for j, mnj in enumerate(model_names):
        if i <= j:
            cm[i][j] = np.nan
        else:
            cm[i][j] = delong_roc_test(ALL_results[task], ALL_results[mni], ALL_results[mnj])[0][0]
cm = pd.DataFrame(cm[1:, :-1], index=model_names[1:], columns=model_names[:-1])
draw_matrix(cm, annot=True, cmap='jet_r', cbar=True)
plt.title(f'Cohort {subset} Delong')
plt.savefig(f'human_AI/external_ibd_normal/delong_each_cohort_{subset}_AI.svg', bbox_inches = 'tight')
plt.show()

In [ ]:
import seaborn as sns
c = pd.concat([a, b], axis=0)
c = c[(c['Signature'] != 'AI')]

# sns.catplot(data=titanic, x="sex", y="survived", hue="class", kind="point")
for m in ['AUC', 'Accuracy', 'Sensitivity', 'Specificity']:
    ax = sns.catplot(
        data=c.reset_index(), kind="point",
        x="Cohort", y=m,
        hue="group"
    )
    ax.set_xlabels(visible=False)# .set_visible(False)
    plt.ylim(0.5 if m != 'Sensitivity' else 0.4)
    from matplotlib.ticker import FormatStrFormatter
    ax.ax.yaxis.set_major_formatter(FormatStrFormatter('%.2f'))
    plt.savefig(f'human_AI\external_ibd_normal\human_boost_{m}.svg', bbox_inches='tight')
    plt.show()